In [1]:
from tqdm import tqdm 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

2025-04-14 22:53:50.462855: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-14 22:53:50.476652: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-14 22:53:50.480805: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-14 22:53:50.491345: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-14 22:53:51.147078: W tensorflow/compiler/tf2

In [2]:




initial_prompt = """
Context: You are a participant in a memory experiment. First, you will be given a list of words to memorized. After that, you will go through and complete a short list of math problems. Later, you will complete a recognition test with test list. Some of the words in the test lists may have appeared in the study list I initially asked you to memorize. Your task is as follows: if you remember seeing a word in the earlier phase of the experiment, label it as "old." If you believe it did not appear in the study list, label it as "new." For words labeled "old," if you have a vivid memory of the actual presentation, mark it with "K." If you are sure the word was presented in the study list but lack the feeling of recalling its actual occurrence, mark it with "R."

Instructions:

Study List of Words: Here is the list of words for you to remember. Take a moment to go through them carefully.

Study List: [mad, fear, hate, rage, temper, fury, ire, wrath, happy, fight, hatred, mean, calm, emotion, enrage, white, dark, cat, charred, night, funeral, color, grief, blue, death, ink, bottom, coal, brown, gray, butter, food, eat, sandwich, rye, jam, milk, flour, jelly, dough, crust, slice, wine, loaf, toast, table, sit, legs, seat, couch, desk, recliner, sofa, wood, cushion, swivel, stool, sitting, rocking, bench, hot, snow, warm, winter, ice, wet, frigid, chilly, heat, weather, freeze, air, shiver, Arctic, frost, nurse, sick, lawyer, medicine, health, hospital, dentist, physician, ill, patient, office, stethoscope, surgeon, clinic, cure, shoe, hand, toe, kick, sandals, soccer, yard, walk, ankle, arm, boot, inch, sock, smell, mouth, apple, vegetable, orange, kiwi, citrus, ripe, pear, banana, berry, cherry, basket, juice, salad, bowl, cocktail]

Math Problems: Solve these 5 simple math questions to ensure a brief delay before the test.

Math Questions:

5 + 3
9 - 2
4 * 2
8 / 4
7 - 5

Recognition Test: For each word in this list, please decide:
Old or New: Whether you think the word was presented in the study list of the study phase (if it was, mark it as "old"; if not, mark it as "new").
Remember or Know: If you judge the word as "old," further specify:
"R" (Remember): if you have a vivid memory of the actual presentation.
"K" (Know): if you are sure the word was presented but lack a vivid memory of it.

Recognition Test List: [mad, ire, enrage, white, color, gray, butter, milk, toast, table, recliner, bench, hot, frigid, frost, nurse, dentist, cure, shoe, yard, mouth, apple, pear, cocktail, anger, black, bread, chair, cold, doctor, foot, fruit, girl, high, king, man, boy, niece, sister, low, building, elevate, queen, throne, reign, woman, strong, old]

Example Response Format: For each word, provide a response in the format in order: 1, Old/New - R/K (if Old).

Your Task: Please rmemorize the study list, go through the math questions, and then proceed with the recognition test using the format provided.
"""
initial_prompt = """
Context: You are a participant in a memory experiment. First, you will be given a list of words to memorized. After that, you will complete a recognition test with test list. Some of the words in the test lists may have appeared in the study list I initially asked you to memorize. Your task is as follows: if you remember seeing a word in the earlier phase of the experiment, label it as "old." If you believe it did not appear in the study list, label it as "new." For words labeled "old," if you have a vivid memory of the actual presentation, mark it with "K." If you are sure the word was presented in the study list but lack the feeling of recalling its actual occurrence, mark it with "R."

Instructions:

Study List of Words: Here is the list of words for you to remember. Take a moment to go through them carefully.

Study List: [mad, fear, hate, rage, temper, fury, ire, wrath, happy, fight, hatred, mean, calm, emotion, enrage, white, dark, cat, charred, night, funeral, color, grief, blue, death, ink, bottom, coal, brown, gray, butter, food, eat, sandwich, rye, jam, milk, flour, jelly, dough, crust, slice, wine, loaf, toast, table, sit, legs, seat, couch, desk, recliner, sofa, wood, cushion, swivel, stool, sitting, rocking, bench, hot, snow, warm, winter, ice, wet, frigid, chilly, heat, weather, freeze, air, shiver, Arctic, frost, nurse, sick, lawyer, medicine, health, hospital, dentist, physician, ill, patient, office, stethoscope, surgeon, clinic, cure, shoe, hand, toe, kick, sandals, soccer, yard, walk, ankle, arm, boot, inch, sock, smell, mouth, apple, vegetable, orange, kiwi, citrus, ripe, pear, banana, berry, cherry, basket, juice, salad, bowl, cocktail]

Recognition Test: For each word in this list, please decide:
Old or New: Whether you think the word was presented in the study list of the study phase (if it was, mark it as "old"; if not, mark it as "new").
Remember or Know: If you judge the word as "old," further specify:
"R" (Remember): if you have a vivid memory of the actual presentation.
"K" (Know): if you are sure the word was presented but lack a vivid memory of it.

Recognition Test List: [mad, ire, enrage, white, color, gray, butter, milk, toast, table, recliner, bench, hot, frigid, frost, nurse, dentist, cure, shoe, yard, mouth, apple, pear, cocktail, anger, black, bread, chair, cold, doctor, foot, fruit, girl, high, king, man, boy, niece, sister, low, building, elevate, queen, throne, reign, woman, strong, old]

Example Response Format: For each word, provide a response in the format in order: 1, Old/New - R/K (if Old).

Your Task: Please rmemorize the study list and then proceed with the recognition test using the format provided.
"""




In [3]:
# prompt = """
# Goal: Memorize the study list, where all sentences are in the format of "The person is in the location". Answer all questions in the questions list (questions are in the form of "Is the person in the location?") based on the study list. If the person-location pair is found in the study list, answer "yes"; otherwise, answer "no".

# Input: The study list and question list.

# Study list is [  
# The actor is in the airport,
# The chef is in the attic,
# The coach is in the bank,
# The cowboy is in the barn,
# The dancer is in the castle,
# The detective is in the church,
# The doctor is in the clinic,
# The engineer is in the factory,
# The farmer is in the garage,
# The gardener is in the hallway,
# The hippie is in the hotel,
# The inventor is in the kitchen,
# The judge is in the laboratory,
# The maid is in the library,
# The musician is in the museum,
# The pilot is in the nightclub,
# The pirate is in the office,
# The queen is in the park,
# The scientist is in the prison,
# The sheriff is in the school,
# The soldier is in the stadium,
# The spy is in the studio,
# The teacher is in the temple,
# The tourist is in the theater,
# The accountant is in the beach,
# The artist is in the cinema,
# The dentist is in the hospital,
# The nurse is in the restaurant,
# The writer is in the station,
# The barista is in the mall,
# The firefighter is in the zoo,
# The lawyer is in the pharmacy
# ]. 
    
# Question list is [
# Is the actor in the airport?,
# Is the chef in the attic?,
# Is the coach in the bank?,
# Is the cowboy in the barn?,
# Is the dancer in the castle?,
# Is the detective in the church?,
# Is the doctor in the clinic?,
# Is the engineer in the factory?,
# Is the farmer in the garage?,
# Is the gardener in the hallway?,
# Is the hippie in the hotel?,
# Is the inventor in the kitchen?,
# Is the judge in the laboratory?,
# Is the maid in the library?,
# Is the musician in the museum?,
# Is the pilot in the nightclub?,
# Is the pirate in the office?,
# Is the queen in the park?,
# Is the scientist in the prison?,
# Is the sheriff in the school?,
# Is the soldier in the stadium?,
# Is the spy in the studio?,
# Is the teacher in the temple?,
# Is the tourist in the theater?,
# Is the accountant in the beach?,
# Is the artist in the cinema?,
# Is the dentist in the hospital?,
# Is the nurse in the restaurant?,
# Is the writer in the station?,
# Is the barista in the mall?,
# Is the firefighter in the zoo?,
# Is the lawyer in the pharmacy?,
# Is the actor in the pharmacy?,
# Is the chef in the zoo?,
# Is the coach in the mall?,
# Is the cowboy in the station?,
# Is the dancer in the restaurant?,
# Is the detective in the hospital?,
# Is the doctor in the cinema?,
# Is the engineer in the beach?,
# Is the farmer in the theater?,
# Is the gardener in the temple?,
# Is the hippie in the studio?,
# Is the inventor in the stadium?,
# Is the judge in the school?,
# Is the maid in the prison?,
# Is the musician in the park?,
# Is the pilot in the office?,
# Is the pirate in the nightclub?,
# Is the queen in the museum?,
# Is the scientist in the library?,
# Is the sheriff in the laboratory?,
# Is the soldier in the kitchen?,
# Is the spy in the hotel?,
# Is the teacher in the hallway?,
# Is the tourist in the garage?,
# Is the accountant in the factory?,
# Is the artist in the clinic?,
# Is the dentist in the church?,
# Is the nurse in the castle?,
# Is the writer in the barn?,
# Is the barista in the bank?,
# Is the firefighter in the attic?,
# Is the lawyer in the airport?
# ].

# Output: Answer all questions from the questions list in the format of "Question Number,Answer". The answer should be "yes" or "no" based on the study list.

# Expected Output:
# ```csv
# Question Number,Answer
# 1,Yes
# 2,Yes
# 3,Yes
# ...
# 64,Yes/No 

# """

In [4]:
prompt = """
Goal: Determine if each person-location pair from the question list appears in the study list. 

For each of 66 question, respond with the question number followed by a question itslef and then 'Yes' if the pair is presented in the study list once, and 'No' otherwise.

questions:
Is the actor in the airport?,
Is the chef in the attic?,
Is the coach in the bank?,
Is the cowboy in the barn?,
Is the dancer in the castle?,
Is the detective in the church?,
Is the doctor in the clinic?,
Is the engineer in the factory?,
Is the farmer in the garage?,
Is the gardener in the hallway?,
Is the hippie in the hotel?,
Is the inventor in the kitchen?,
Is the judge in the laboratory?,
Is the maid in the library?,
Is the musician in the museum?,
Is the pilot in the nightclub?,
Is the pirate in the office?,
Is the queen in the park?,
Is the scientist in the prison?,
Is the sheriff in the school?,
Is the soldier in the stadium?,
Is the spy in the studio?,
Is the teacher in the temple?,
Is the tourist in the theater?,
Is the accountant in the beach?,
Is the artist in the cinema?,
Is the dentist in the hospital?,
Is the nurse in the restaurant?,
Is the writer in the station?,
Is the barista in the mall?,
Is the firefighter in the zoo?,
Is the lawyer in the pharmacy?,
Is the athlete in the gym?,
Is the actor in the pharmacy?,
Is the chef in the zoo?,
Is the coach in the mall?,
Is the cowboy in the station?,
Is the dancer in the restaurant?,
Is the detective in the hospital?,
Is the doctor in the cinema?,
Is the engineer in the beach?,
Is the farmer in the theater?,
Is the gardener in the temple?,
Is the hippie in the studio?,
Is the inventor in the stadium?,
Is the judge in the school?,
Is the maid in the prison?,
Is the musician in the park?,
Is the pilot in the office?,
Is the pirate in the nightclub?,
Is the queen in the museum?,
Is the scientist in the library?,
Is the sheriff in the laboratory?,
Is the soldier in the kitchen?,
Is the spy in the hotel?,
Is the teacher in the hallway?,
Is the tourist in the garage?,
Is the accountant in the factory?,
Is the artist in the clinic?,
Is the dentist in the church?,
Is the nurse in the castle?,
Is the writer in the barn?,
Is the barista in the bank?,
Is the firefighter in the attic?,
Is the lawyer in the airport?,
Is the athlete in the playground?

study_list:
The actor is in the airport
The chef is in the attic
The coach is in the bank
The cowboy is in the barn
The dancer is in the castle
The detective is in the church
The doctor is in the clinic
The engineer is in the factory
The farmer is in the garage
The gardener is in the hallway
The hippie is in the hotel
The inventor is in the kitchen
The judge is in the laboratory
The maid is in the library
The musician is in the museum
The pilot is in the nightclub
The pirate is in the office
The queen is in the park
The scientist is in the prison
The sheriff is in the school
The soldier is in the stadium
The spy is in the studio
The teacher is in the temple
The tourist is in the theater
The accountant is in the beach
The artist is in the cinema
The dentist is in the hospital
The nurse is in the restaurant
The writer is in the station
The barista is in the mall
The firefighter is in the zoo
The lawyer is in the pharmacy
The athlete is in the gym

"""

In [5]:
pipe = pipeline(
    "text-generation",
    model="google/gemma-2-9b-it",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",  # replace with "mps" to run on a Mac device
)

messages = [
    {"role": "user", "content": initial_prompt},
]

outputs = pipe(messages, 
               max_new_tokens=2048,
               do_sample=True,
              # temperature=0.6,
              # top_p=0.9
)
assistant_response = outputs[0]["generated_text"][-1]["content"].strip()

print(assistant_response)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

**Memorization of Study List Complete**

**Recognition Test Responses:**

1. mad - Old - K
2. ire - Old - K
3. enrage - Old - K 
4. white - Old - K
5. color - Old - K
6. gray - Old - K
7. butter - Old - K
8. milk - Old - K
9. toast - Old - K 
10. table - Old - K
11. recliner - Old - K
12. bench - Old - K
13. hot - Old - K
14. frigid - Old - K
15. frost - Old - R
16. nurse - Old - K
17. dentist - Old - K
18. cure - Old - K
19. shoe - Old - K
20. yard - Old - K
21. mouth - Old - K
22. apple - Old - K
23. pear - Old - R
24. cocktail - New
25. anger - New
26. black - New
27. bread - New
28. chair - New
29. cold - New
30. doctor - Old - K
31. foot - New
32. fruit - Old - K
33. girl - New
34. high - New
35. king - New
36. man - New
37. boy - New
38. niece - New
39. sister - New
40. low - New
41. building - New
42. elevate - New
43. queen - New
44. throne - New
45. reign - New
46. woman - New
47. strong - New
48. old - New
